# Vowpal Wabbit в NLP
Автоматическая обработка текстов - 2017, семинар 4.

В этом семинаре мы познакомимся с библиотекой Vowpal Wabbit и решим с его помощью задачу многоклассовой классификации на больших данных. 
Данные скачайте [здесь](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data) или запустите следующие две ячейки.

Чтобы на семинаре не тратить время на обработку и обучение моделей на всех данных, предлагается использовать только небольшую подвыборку (`train-sample.csv`). Но сдавать ноутбук все равно необходимо с результатами на **всех** данных.

In [1]:
import numpy as np

In [2]:
import csv

INPUT_DATA = 'train.csv'
# INPUT_DATA = 'train-sample.csv'

reader = csv.DictReader(open(INPUT_DATA))
dict(next(reader))

{'BodyMarkdown': "I'm new to C#, and I want to use a trackbar for the forms opacity\nThis is my code\n\n    decimal trans = trackBar1.Value / 5000\n    this.Opacity = trans\n\nWhen I try to build it, I get this error\n\n**Cannot implicitly convert type 'decimal' to 'double**\n\nI tried making trans a double, but then the control doesn't work. This code worked fine for me in VB.NET. Any suggestions?",
 'OpenStatus': 'open',
 'OwnerCreationDate': '07/31/2008 21:33:24',
 'OwnerUndeletedAnswerCountAtPostTime': '0',
 'OwnerUserId': '8',
 'PostClosedDate': '',
 'PostCreationDate': '07/31/2008 21:42:52',
 'PostId': '4',
 'ReputationAtPostCreation': '1',
 'Tag1': 'c#',
 'Tag2': '',
 'Tag3': '',
 'Tag4': '',
 'Tag5': '',
 'Title': 'Decimal vs Double?'}

Каждый объект выборки соответствует некоторому посту на Stack Overflow. Требуется построить модель, определяющую статус поста. Подробнее про задачу и формат данных можно прочитать на [странице соревнования](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow).

Перед обучением модели из Vowpal Wabbit данные следует сохранить в специальный формат: <br>
`label |namespace1 feature1:value1 feature2 feature3:value3 |namespace2 ...` <br>
Записи `feature` и `feature:1.0` эквивалентны. Выделение признаков в смысловые подгруппы (namespaces) позволяет создавать взаимодействия между ними. Подробнее про формат входных данных можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format).

Ниже реализована функция, которая извлекает признаки с помощью подаваемого на вход экстрактора, разбивает данные на трейн и тест и записывает их на диск.

In [23]:
STATUSES = ['not a real question', 'not constructive', 'off topic', 'open', 'too localized']
STATUS_DICT = {status: i+1 for i, status in enumerate(STATUSES)}

def data2vw(features_extractor, input_file='train.csv',
            train_output='train', test_output='test',
            ytest_output='ytest'):
    reader = csv.DictReader(open(input_file))
    writer_train = open(train_output, 'w')
    writer_test = open(test_output, 'w')
    writer_ytest = open(ytest_output, 'w')
    
    for row in reader:
        label = STATUS_DICT[row['OpenStatus']]
        features = features_extractor(row)
        output_line = '%s %s\n' % (label, features)
        if int(row['PostId']) % 2 == 0:
            writer_train.write(output_line)
        else:
            writer_test.write(output_line)
            writer_ytest.write('%s\n' % label)
            
    writer_train.close()
    writer_test.close()
    writer_ytest.close()

Начнем с простейшей модели. В качестве признаков возьмите заголовки и очистите их: приведите символы к нижнему регистру, удалите пунктуацию. Также приветствуется использование стеммеров/лемматизаторов, однако учтите, что они могут сильно замедлить скорость обработки.

In [ ]:
import re

In [4]:
def extract_title(row):
    title = row['Title'].lower()
    title = re.sub('[^ a-z]', '', title)
    return title


data2vw(lambda row: '| %s' % extract_title(row))

! head -n 5 train

4 | decimal vs double
4 | percentage width child in absolutely positioned parent doesnt work in ie
4 | tools for porting j code to c
4 | throw error in mysql trigger
4 | whats the difference between mathfloor and mathtruncate


Обучим `vw` модель. Параметр `-d` отвечает за путь к обучающей выборке, `-f` – за путь к модели, `--oaa` – за режим мультиклассовой классификации `one-against-all`. Подробное описание всех параметров можно найти [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments) или вызвав `vw --help`.

In [5]:
! vw -d train --loss_function logistic --oaa 5 -f model

final_regressor = model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = train
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4        1        4
0.500000 0.000000            2            2.0        4        4       12
0.250000 0.000000            4            4.0        4        4        6
0.125000 0.000000            8            8.0        4        4        7
0.062500 0.000000           16           16.0        4        4        4
0.062500 0.062500           32           32.0        4        4        9
0.046875 0.031250           64           64.0        4        4        5
0.062500 0.078125          128          128.0        4        4       11
0.089844 0.117188          256          256.0        4        4        8
0.070312 0.050781          512          512.0   

Применим модель к тестовой выборке и сохраним предсказания.

In [6]:
! vw -i model -t test -r pred

only testing
raw predictions = pred
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        4        4        9
0.000000 0.000000            2            2.0        4        4        7
0.000000 0.000000            4            4.0        4        4        8
0.125000 0.250000            8            8.0        4        4        6
0.125000 0.125000           16           16.0        4        4        7
0.156250 0.187500           32           32.0        4        4       10
0.093750 0.031250           64           64.0        4        4        8
0.117188 0.140625          128          128.0        4        4        8
0.125000 0.132812          256          256.0        4        4       15
0.109375 0.093750          512       

In [7]:
! head -n 3 pred

1:-4.35192 2:-5.91851 3:-4.50567 4:2.55003 5:-6.68014
1:-7.03341 2:-7.67406 3:-6.85027 4:4.34531 5:-6.29331
1:-4.26167 2:-5.55281 3:-5.30194 4:2.89059 5:-7.50819


Реализуйте функцию, которая вычисляет `logloss` и `accuracy`, не загружая вектора в память. Используйте `softmax`, чтобы получить вероятности.

In [5]:
def get_scores(ytest_input='ytest', pred_input='pred'):
    n, error, loss = 0, 0, 0
    reader_ytest = open(ytest_input, 'r')
    reader_pred = open(pred_input, 'r')

    for label, pred in zip(reader_ytest, reader_pred):
        n += 1
        label = int(label) - 1
        scores = [float(score.split(':')[1]) for score in pred.split(' ')]
        scores = np.array(scores)
        denominator = np.sum(np.exp(scores))
        scores = np.exp(scores) / denominator
        
        prediction = np.argmax(scores) 
        
        loss += np.log(scores[label])
        if prediction != label:
            error += 1
        
    reader_ytest.close()
    reader_pred.close()
    return - loss / n, 1 - float(error) / n

In [9]:
! vw --help

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1


VW options:
  --random_seed arg                     seed random number generator
  --ring_size arg                       size of example ring

Update options:
  -l [ --learning_rate ] arg            Set learning rate
  --power_t arg                         t power value
  --decay_learning_rate arg             Set Decay factor for learning_rate 
                                        between passes
  --initial_t arg                       initial t value
  --feature_mask arg                    Use existing regressor to determine 
                                        which parameters may be updated.  If no
                                        initial_regressor given, also used for 
                                        initial weights.

Weight options:
  -i [ --initial_regressor ] arg        Initial regressor(s)
  --initial_weight arg                  Set all 

In [10]:
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.14684
accuracy = 0.97793


На оригинальных данных `logloss` должен быть меньше `0.20`, `accuracy` больше `0.95`. Если это не так, то скорее всего у вас ошибка.

Теперь попробуем улучшить модель, добавив новые признаки, порождаемые словами. В `vowpal wabbit` есть возможность делать это прямо на лету. Воспользуйтесь параметрами `affix`, `ngram`, `skips`.

Далее везде при подборе параметров ориентируйтесь на улучшение `logloss`. Используйте `--quiet` или `-P`, чтобы избавиться от длинных выводов при обучении и применении моделей.

In [15]:
! vw -d train --loss_function logistic --oaa 5 -f model --ngram 2 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.17138
accuracy = 0.97581


In [14]:
! vw -d train --loss_function logistic --oaa 5 -f model --ngram 2 --ngram 3 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.18119
accuracy = 0.97577


In [17]:
! vw -d train --loss_function logistic --oaa 5 -f model --skips 1 --ngram 2 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.17947
accuracy = 0.97561


In [18]:
! vw -d train --loss_function logistic --oaa 5 -f model --skips 2 --ngram 3 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.16892
accuracy = 0.97699


In [19]:
! vw -d train --loss_function logistic --oaa 5 -f model --skips 3 --ngram 4 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.16337
accuracy = 0.97785


In [20]:
! vw -d train --loss_function logistic --oaa 5 -f model --skips 3 --ngram 4 --ngram 5 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.16697
accuracy = 0.97822


In [21]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+1,-1' --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.13856
accuracy = 0.97847


In [22]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2,-2' --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.13862
accuracy = 0.97835


В итоге только суффиксы дали улучшение, причем длины 1.

Часто качество `vw` модели получается учушить увеличением числа проходов по обучающей выборке (параметр `--passes`) и увеличением числа бит хэш-функции для уменьшения числа коллизий признаков (параметр `-b`). Подробнее про то, где в `vowpal wabbit` используется хэш-функция, можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Feature-Hashing-and-Extraction). Как меняется качество при изменении этих параметров? Верно ли, что при увеличении значений параметров `--passes` и `-b` качество всегда не убывает и почему?

In [25]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2,-2' --quiet --passes 5 --cache_file cache_file
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.13940
accuracy = 0.97839


In [26]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2,-2' --quiet --passes 3 --cache_file cache_file
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.13771
accuracy = 0.97815


In [27]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2,-2' --quiet --passes 3 --cache_file cache_file -b 10
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.16325
accuracy = 0.97927


In [28]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2,-2' --quiet --passes 3 --cache_file cache_file -b 20
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.13663
accuracy = 0.97807


In [10]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2,-2' --quiet --passes 3 --cache_file cache_file -b 25
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.13642
accuracy = 0.97801


При увеличении passes результат вполне может и ухудшится, ведь при градиентном спуске нет гарантий, что лосс будет всегда убывать, мы знаем только, что он сойдется к какому-то локальному минимуму. При увеличении размера таблицы фичей мы можем столкнуться с недообучением, если сделаем её слишком большой (это, конечно, если у нас очень много признаков), так что тоже не не факт, что лосс всегда будет уменьшаться.

Теперь интерес представляет то, какие признаки оказались наиболее важными для модели. Для этого сначала переведем модель в читаемый формат.

In [11]:
! vw -i model -t --invert_hash model.readable train --quiet
! head -n 30 model.readable

Version 8.1.1
Min label:-50.000000
Max label:50.000000
bits:25
lda:0
0 ngram: 
0 skip: 
options: --affix +2,-2 --oaa 5
Checksum: 1485410803
:0
Constant:26094304:-2.752298
Constant[1]:26094305:-3.298096
Constant[2]:26094306:-3.059249
Constant[3]:26094307:2.244808
Constant[4]:26094308:-3.530444
a:19615120:-0.021666
a[1]:19615121:0.075131
a[2]:19615122:-0.016544
a[3]:19615123:0.007759
a[4]:19615124:-0.008260
aa:8686920:-1.165970
aa[1]:8686921:-1.022695
aa[2]:8686922:-0.967443
aa[3]:8686923:1.049916
aa[4]:8686924:-0.609711
aaa:8584632:-0.167988
aaa[1]:8584633:-0.472787
aaa[2]:8584634:-0.640633
aaa[3]:8584635:0.214563
aaa[4]:8584636:-0.298246


Первые несколько строк соответствуют информации о модели. Далее следуют строчки вида `feature[label]:hash:weight`. Выделите для каждого класса 10 признаков с наибольшими по модулю весами. Постарайтесь сделать ваш алгоритм прохода по файлу константным по памяти. Например, можно воспользоваться [кучей](https://docs.python.org/2/library/heapq.html).

In [6]:
from collections import namedtuple

- Определяем namedtuple Feature, содержащий имя, хэш и вес
- Пишем функцию, которая из строчки достает Feature и label.
- Пишем функцию, которая сравнивает два экземпляра Feature.
- Пишем класс, который позволяет хранить топ объектов: он представляет из себя лист, в который элементы добавляются по убыванию, причем если их число превышает $n$ (передаваемое в конструкторе), то лист обрезается до $n$. Таким образом, память константа.

In [7]:
Feature = namedtuple('Feature', ['name', 'hash', 'weight'])


def get_feature_and_label(row):
    row = row.strip().split(':')
    if len(row) != 3:
        return None, None
    
    ind = row[0].find('[')
    if ind == -1:
        label = 0
        feature_name = row[0]
    else:
        label = int(row[0][ind + 1])
        feature_name = row[0][:ind]

    feature_hash = int(row[1])
    feature_weight = float(row[2])
    
    return Feature(feature_name, feature_hash, feature_weight), label


def compare(a, b):
    return abs(a.weight) < abs(b.weight)


class Top:
    def __init__(self, n, compare):
        """
        n - the max number element in top
        compare(a, b) should represent a < b
        """
        self._n = n
        self._top = []
        self._compare = compare
        
    def _insert(self, i, elem):
        self._top.insert(i, elem)
        if len(self._top) > self._n:
                self._top.pop()

    def add(self, elem):
        for i in range(len(self._top) - 1, -1, -1):
            if self._compare(elem, self._top[i]):
                self._insert(i + 1, elem)
                break
        else:
            self._insert(0, elem)

    def get_top(self):
        return self._top

In [31]:
most_important = [Top(10, compare) for i in range(5)]
with open('model.readable') as features_file:
    for line in features_file:
        feature, label = get_feature_and_label(line)
        if feature is not None:
            most_important[label].add(feature)

In [32]:
for status, top_features in zip(STATUSES, most_important):
    print(status)
    for feature in top_features.get_top():
        print(feature)

not a real question
Feature(name='mercurial', hash=32544120, weight=-4.177212)
Feature(name='firing', hash=6468328, weight=-4.052338)
Feature(name='uitableviewcell', hash=20184896, weight=-3.990192)
Feature(name='activerecord', hash=5782720, weight=-3.91699)
Feature(name='disabling', hash=10749160, weight=-3.89928)
Feature(name='launching', hash=4314656, weight=-3.781238)
Feature(name='subquery', hash=32226568, weight=-3.725806)
Feature(name='groups', hash=30711456, weight=-3.6696)
Feature(name='centering', hash=22058864, weight=-3.639237)
Feature(name='forward', hash=12034776, weight=-3.481619)
not constructive
Feature(name='property', hash=5748833, weight=-3.546955)
Feature(name='Constant', hash=26094305, weight=-3.298096)
Feature(name='staticsized', hash=30737089, weight=-3.256423)
Feature(name='match', hash=30737089, weight=-3.256423)
Feature(name='webview', hash=22162041, weight=-3.187493)
Feature(name='jqgrid', hash=8268369, weight=-3.113597)
Feature(name='jar', hash=12536961, we

Добавим признаки, извлеченные из текста поста (поле `BodyMarkdown`). В этом поле находится более подробная информация о вопросе, и часто туда помещают код, формулы и т.д. При удалении пунктуации мы потеряем много полезной информации, однако модель "мешка слов" на сырых данных может сильно раздуть признаковое пространство. В таких случаях работают с n-граммами на символах. <br>
Будьте осторожны: символы "`:`" и "`|`" нельзя использовать в названиях признаков, поскольку они являются служебными для `vw`-формата. Замените эти символы на два других редко встречающихся в выборке (или вообще не встречающихся). Также не забудьте про "`\n`". <br>
Поскольку для каждого документа одна n-грамма может встретиться далеко не один раз, то будет экономнее записывать признаки в формате `[n-грамма]:[число вхождений]`.

Также добавьте тэги (поля вид `TagN`). Приветствуется добавление информации о пользователе из других полей. Только не используйте `PostClosedDate` – в нем содержится информация о таргете.

In [8]:
from collections import Counter

In [75]:
def extract_ngram_body(row, ngram=3):
    body = row['BodyMarkdown'].strip().lower()
    body = re.sub(':', 'ю', body)
    body = re.sub('\|', 'ф', body)
    body = re.sub('\n+', ' ', body)
    
    ngrams = [word[i:i+ngram]
              for word in body.split(' ')
              for i in range(len(word) - ngram + 1)]
    counter = Counter(ngrams)
    features = [ngram + ':' + str(cnt)
                for ngram, cnt in counter.items()]
    return ' '.join(features)

def extract_tags(row):
    i = 1
    tags = []
    while row.get('Tag' + str(i)):
        tag = row['Tag' + str(i)]
        if len(tag) > 0:
            tags.append(tag)
        i += 1
    
    return ' '.join(tags)

Объединим все вместе. Реализуйте экстрактор признаков, который выделяет каждую подгруппу в отдельный namespace.

In [76]:
extractors_list = [
    ('t', extract_title), 
    ('b', extract_ngram_body), 
    ('a', extract_tags)
] # (namespace, extractor)


def make_feature_extractor(extractors_list):
    def feature_extractor(row):
        features = ['|' + namespace + ' ' + extractor(row)
                    for namespace, extractor in extractors_list]
        return ' '.join(features)
    return feature_extractor

In [77]:
data2vw(make_feature_extractor(extractors_list))

! head -n 1 train

4 |t decimal vs double |b i'm:1 new:1 c#,:1 and:1 wan:1 ant:1 use:1 tra:5 rac:2 ack:2 ckb:2 kba:2 bar:2 for:3 the:3 orm:1 rms:1 opa:2 pac:2 aci:2 cit:3 ity:2 thi:4 his:4 cod:2 ode:2 dec:2 eci:2 cim:2 ima:2 mal:2 ran:3 ans:3 ar1:1 r1.:1 1.v:1 .va:1 val:1 alu:1 lue:1 500:1 000:1 is.:1 s.o:1 .op:1 whe:1 hen:2 try:1 bui:1 uil:1 ild:1 it,:1 get:1 err:1 rro:1 ror:1 **c:1 *ca:1 can:1 ann:1 nno:1 not:1 imp:1 mpl:1 pli:1 lic:1 ici:1 itl:1 tly:1 con:2 onv:1 nve:1 ver:1 ert:1 typ:1 ype:1 'de:1 al':1 'do:1 dou:2 oub:2 ubl:2 ble:2 le*:1 e**:1 tri:1 rie:1 ied:1 mak:1 aki:1 kin:1 ing:1 le,:1 but:1 ont:1 ntr:1 tro:1 rol:1 doe:1 oes:1 esn:1 sn':1 n't:1 wor:2 ork:2 rk.:1 rke:1 ked:1 fin:1 ine:1 vb.:1 b.n:1 .ne:1 net:1 et.:1 any:1 sug:1 ugg:1 gge:1 ges:1 est:1 sti:1 tio:1 ion:1 ons:1 ns?:1 |a c#


In [78]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 --cache_file cache_file -b 25
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.32839
accuracy = 0.97926


DEBUG BEGIN

In [73]:
data2vw(make_feature_extractor(extractors_list), input_file='train-sample.csv',
        train_output='train_small', test_output='test_small',
        ytest_output='ytest_small')

! head -n 1 train_small

4 |t for mongodb is it better to reference an object or use a natural string key |b bui:1 uil:1 ild:1 ldi:1 din:1 ing:3 cor:1 orp:1 rpu:1 pus:1 ind:1 nde:1 dex:1 exe:1 xed:1 sen:2 ent:3 nte:2 ten:2 enc:4 nce:3 ces:2 dif:1 iff:1 ffe:1 fer:3 ere:4 ren:3 lan:5 ang:5 ngu:5 gua:5 uag:5 age:5 ges:2 es.:1 hav:2 ave:2 col:3 oll:3 lle:3 lec:3 ect:5 cti:5 tio:3 ion:3 whi:1 hic:1 ich:1 bot:1 oth:1 obj:2 bje:2 jec:2 tid:1 and:1 the:6 iso:1 cod:1 ode:1 key:2 ey.:1 bet:2 ett:1 tte:1 ter:1 use:1 ref:2 efe:2 sto:1 tor:1 ore:1 lik:1 ike:1 "en:1 en":1 "fr:1 fr":1 r"?:1 "?i:1 sup:1 upp:1 ppo:1 pos:1 ose:1 it':1 t's:1 com:1 omp:1 mpr:1 pro:1 rom:1 omi:1 mis:1 ise:1 etw:1 twe:1 wee:1 een:1 enю:1 eas:1 ase:1 nci:1 cin:1 tha:2 hat:2 spe:1 pee:1 eed:1 doi:1 oin:1 que:1 uer:1 eri:1 rie:1 ies:1 whe:1 her:1 has:1 cer:1 ert:1 rta:1 tai:1 ain:1 siz:1 ize:1 dat:1 ata:1 dis:1 isk:1 ska:1 kan:1 any:1 bes:1 est:1 pra:1 rac:1 act:1 tic:1 ice:1 sho:1 hou:1 oul:1 uld:1 kno:1 now:1 of?:1 |a mongodb


In [74]:
! vw -d train_small --loss_function logistic --oaa 5 -f model_small --affix '+2t,-2t' -b 25 --quite
! vw -i model_small -t test_small -r pred
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

final_regressor = model_small
Num weight bits = 25
learning rate = 0.5
initial_t = 0
power_t = 0.5
vw: unrecognised option '--quite'
only testing
raw predictions = pred
Num weight bits = 25
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test_small
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        4        4      352
0.000000 0.000000            2            2.0        4        4       93
0.250000 0.500000            4            4.0        4        4      221
0.375000 0.500000            8            8.0        3        1      102
0.312500 0.250000           16           16.0        4        4       52
0.375000 0.437500           32           32.0        4        4      258
0.390625 0.406250           64           64.0        3        3      122
0.335938 0.281250          128          128.

DEBUG END

Поэкспериментируйте с другими параметрами модели. Добавьте квадратичные взаимодействия между различными блоками признаков, измените параметры оптимизатора, добавьте регуляризацию и т.д. Совсем необязательно перебирать все параметры по сетке и добиваться оптимального качества. Для нас важнее то, насколько хорошо вы разобрались с возможностями библиотеки и продемонстрировали это.

Выберите не менее трех параметров. Для каждого из них объясните, почему по вашему мнению его изменение может улучшить качество модели, подберите оптимальное значение. Можете перебрать несколько значений "руками", а можете воспользоваться [vw-hypersearch](https://github.com/JohnLangford/vowpal_wabbit/wiki/Using-vw-hypersearch) или [vw-hyperopt](https://github.com/JohnLangford/vowpal_wabbit/blob/master/utl/vw-hyperopt.py) ([статья на хабре](https://habrahabr.ru/company/dca/blog/272697/)). Какие параметры повлияли на улучшение качества сильнее всего?

-l, --decay_learning_rate (0,5, 0,75, 1), -q, --l1, --l2, --power_t (0, 1/2, 1)

Подбор расписания learning rate часто может существенно повлиять на успешность обучения, из-за нетривиальных функций, которые мы пытаемся оптимизировать. Поэтому попробуем поварьировать -l, --decay_learning_rate и --power_t. Так же в виду большого количества признаков многие из них неинформативы, поэтому необходимо использовать --l1, чтобы не переобучиться и чтобы отсеять несущественные. Ну и заодно можно попробовать поварьировать и --l2, ведь вычисления с ней по крайней мере быстрые)

Так же попробуем -q. -q bb - слишком долго, попробуем -q tb.

In [89]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 -q tb
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.18073
accuracy = 0.97927


In [90]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 -l 0.05
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.13734
accuracy = 0.97926


In [91]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 -l 0.005
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.42243
accuracy = 0.97927


In [92]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 -l 0.0005
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.16870
accuracy = 0.97927


самым эффективным окзался дефолтный -l

In [93]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.1
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.60944
accuracy = 0.00904


In [94]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.01
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.60944
accuracy = 0.00904


In [95]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.60944
accuracy = 0.00904


In [100]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.0001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.60944
accuracy = 0.00904


In [99]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.00001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.12815
accuracy = 0.97927


In [101]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.000001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.15469
accuracy = 0.97927


In [96]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l2 0.1
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.60914
accuracy = 0.97927


In [97]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l2 0.01
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.60656
accuracy = 0.97927


In [98]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l2 0.001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.57784
accuracy = 0.97927


In [102]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l2 0.0001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.31926
accuracy = 0.97927


In [103]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l2 0.00001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.70489
accuracy = 0.97901


In [98]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l2 0.0001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.57784
accuracy = 0.97927


In [104]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l2 0.00001
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.70489
accuracy = 0.97901


l2 оказалась бесполезной, а вот l1 очень неплохо уменьшила лосс

Давайте теперь подберем расписание learning rate'a. С дефолтными $decay = 1$ и $power\_t = 0$ $loss = 0.128$

In [106]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.00001 --power_t 0.5
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.12815
accuracy = 0.97927


In [107]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.00001 --power_t 1
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.58016
accuracy = 0.01690


In [108]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.00001 --decay_learning_rate 0.75
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.12795
accuracy = 0.97927


In [109]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.00001 --decay_learning_rate 0.75 --power_t 0.5
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.12795
accuracy = 0.97927


In [110]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.00001 --decay_learning_rate 0.75 --power_t 1
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.57922
accuracy = 0.01690


In [111]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.00001 --decay_learning_rate 0.5
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.12757
accuracy = 0.97927


In [112]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.00001 --decay_learning_rate 0.5 --power_t 0.5
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 0.12757
accuracy = 0.97927


In [113]:
! vw -d train --loss_function logistic --oaa 5 -f model --affix '+2t,-2t' --quiet --passes 3 \
--cache_file cache_file -b 25 --l1 0.00001 --decay_learning_rate 0.5 --power_t 1
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

malformed example!
words.size() = 7342
logloss  = 1.33554
accuracy = 0.03956


--l1 0.00001 очень сильно улучшил модель. До регуляризации результаты были хуже, чем у модели, опиравшейся только на title. После добавления регуляризации лосс уменьшился в 3 раза и стал меньше, чем был у простой модели. Расписание learning rate помогло не так сильно, но тоже сумело немного уменьшить лосс.

В итоге лучшие параметры: --affix '+2t,-2t' --passes 3 -b 25 --l1 0.00001 --decay_learning_rate 0.5 --power_t 0.5